# Advanced Machine Learning
## Deep Reinforcement Learning

&copy; Helena Aidos, 2024

### 3. Train the DQN agent

In this part of the project we will be performing **Deep Reinforcement Learning using JAX**, a library for array-oriented computation that enables high-performance machine learning research. Although Deep Reinforcement Learning can be computationally expensive and data-intensive, it is a powerful framework for solving problems where explicit programming or rule-based approaches fall short.

Similarly to Q-learning, DQN agents are defined by two functions: `act` and `update`.
- `act` function: implements an epsilon-greedy policy with respect to Q-values and the agent selects the action corresponding to the **maximum predicted Q-value** for a given state, with a set probability of acting randomly.
- `update` function: responsible for training the network: $$L(\theta)=E\left[ (r+(1-done) \times \gamma \max_{a'} Q(s',a';\theta^-) - Q(s,a;\theta))^2 \right],$$ where $\theta$ denotes the **parameters of the online network** and $\theta^-$ represents the **parameters of the target network**. Training is stabilized due to this separation of parameters.


#### 3.1 Creating a Gymnax environment: Mountain Car

We will be using `gymnax`as it is ideal for applications like meta-reinforcement learning. It provides precise functional control over environment settings, such as random seeds and hyperparameters, and enables parallelized rollouts across different configurations.

In [642]:
import gymnax

In [643]:
import jax

rng = jax.random.PRNGKey(0)
rng, key_reset, key_policy, key_step = jax.random.split(rng, 4)

# create the Mountain Car environment
env, env_params = gymnax.make('MountainCar-v0')
# inspect default environment settings
env_params

EnvParams(max_steps_in_episode=200, min_position=-1.2, max_position=0.6, max_speed=0.07, goal_position=0.5, goal_velocity=0.0, force=0.001, gravity=0.0025)

In [644]:
# observation space dimensionality
env.observation_space(env_params).shape

(2,)

In [645]:
# action space dimensionality
env.action_space(env_params).n

3

In [646]:
# alternatively, action space dimensionality
env.num_actions

3

In [647]:
obs, state = env.reset(key_reset, env_params)
obs, state

(Array([-0.58275205,  0.        ], dtype=float32),
 EnvState(time=Array(0, dtype=int32, weak_type=True), position=Array(-0.58275205, dtype=float32), velocity=Array(0., dtype=float32, weak_type=True)))

In [648]:
action = env.action_space(env_params).sample(key_policy)
n_obs, n_state, reward, done, _ = env.step(key_step, state, action, env_params)
n_obs, n_state, reward, done

C:\Users\jessi\AppData\Roaming\Python\Python312\site-packages\jax\_src\numpy\array_methods.py:118: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  return lax_numpy.astype(self, dtype, copy=copy, device=device)


(Array([-5.8331072e-01, -5.5867527e-04], dtype=float32),
 EnvState(time=Array(1, dtype=int32, weak_type=True), position=Array(-0.5833107, dtype=float32), velocity=Array(-0.00055868, dtype=float32)),
 Array(-1., dtype=float32, weak_type=True),
 Array(False, dtype=bool))

You can also simply use the environment with its default settings:

In [650]:
obs, state = env.reset(key_reset)
action = env.action_space().sample(key_policy)
n_obs, n_state, reward, done, _ = env.step(key_step, state, action)

C:\Users\jessi\AppData\Roaming\Python\Python312\site-packages\jax\_src\numpy\array_methods.py:118: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  return lax_numpy.astype(self, dtype, copy=copy, device=device)


`gymnax` provides fully functional environment dynamics that can leverage the full power of JAX's function transformations. E.g. one common RL use-case the parallel rollout of multiple workers. Using a `vmap` across random seeds (one per worker) allows us to implement such a parallelization on a single machine:

In [652]:
vmap_reset = jax.vmap(env.reset, in_axes=(0, None))
vmap_step = jax.vmap(env.step, in_axes=(0, 0, 0, None))

num_envs = 8
vmap_keys = jax.random.split(rng, num_envs)

obs, state = vmap_reset(vmap_keys, env_params)
n_obs, n_state, reward, done, _ = vmap_step(vmap_keys, state, jax.numpy.zeros(num_envs), env_params)
print(n_obs.shape)

(8, 2)


#### 3.2. Implementing a DQN agent using RLinJAX

`RLinJAX` is a library of deep reinforcement learning algorithms which will allows to set up, train and evaluate deep RL agents. This library has implemented several deep reinforcement learning algorithms, from which we will be using: `dqn` (deep Q-network) and `sac` (soft actor-critic)


In [654]:
import time
from RLinJAX import *

In [655]:
jax.devices()

[CpuDevice(id=0)]

We will be setting up the training configuration using the following parameters:

In [657]:
CONFIGS = {"dqn":{"agent": "DuelingQNetwork",
                  "agent_kwargs": {"activation": "swish"},
                  #"num_envs": 10,
                  "buffer_size": 50_000,
                  "fill_buffer": 10_000,
                  "batch_size": 100,
                  "learning_rate": 0.001,
                  "polyak": 0.95,
                  "num_epochs": 5,
                  "target_update_freq": 1000,
                  "total_timesteps": 200_000,
                  "eval_freq": 5000,
                  "eps_start": 1,
                  "eps_end": 0.01,
                  "exploration_fraction": 0.3,
                  "gamma": 0.99,
                  "ddqn": True
                 }
          }

We will now create the algorithm instance for DQN in the MountainCar-v0 environment. 

In [659]:
algo_str = "dqn"
env_str = "MountainCar-v0"

# `get_algo` is a convenient function to access algorithms easily
# `alg_cls.create` creates a frozen instance of the algorithm and populates it with default values
algo_cls = get_algo(algo_str)
algo = algo_cls.create(env=env_str, **CONFIGS.get(algo_str, {}))

Let's look at the algorithm instance we have created:

In [661]:
algo.config

{'env': <gymnax.environments.classic_control.mountain_car.MountainCar at 0x1eb52044d40>,
 'env_params': {'max_steps_in_episode': 200,
  'min_position': -1.2,
  'max_position': 0.6,
  'max_speed': 0.07,
  'goal_position': 0.5,
  'goal_velocity': 0.0,
  'force': 0.001,
  'gravity': 0.0025},
 'eval_callback': <function RLinJAX.algos.algorithm.Algorithm.create.<locals>.eval_callback(algo, ts, rng)>,
 'eval_freq': 5000,
 'skip_initial_evaluation': False,
 'total_timesteps': 200000,
 'learning_rate': 0.001,
 'gamma': 0.99,
 'max_grad_norm': inf,
 'normalize_observations': False,
 'target_update_freq': 1000,
 'polyak': 0.95,
 'num_envs': 1,
 'buffer_size': 50000,
 'fill_buffer': 10000,
 'batch_size': 100,
 'eps_start': 1,
 'eps_end': 0.01,
 'exploration_fraction': 0.3,
 'agent': {'hidden_layer_sizes': (64, 64),
  'activation': <PjitFunction>,
  'action_dim': 3,
  'parent': None,
  'name': None},
 'num_epochs': 5,
 'ddqn': True}

In [662]:
# the `replace` method will be used to tune hyperparameters and variables
algo = algo.replace(gamma=0.95)
#algo = algo.replace(agent={"hidden_layer_sizes": (128, 128)})
print(f"New gamma: {algo.gamma}")

New gamma: 0.95


We will now be training the agent:

In [664]:
# set training seed and jit train function. Jit `algo.train` allows for faster execution
rng = jax.random.PRNGKey(0)
train_fn = jax.jit(algo.train)

print("Starting to train")
# Train: the initial train_state holds currents environment step, network parameters and replay buffer contents
start = time.time()
train_state, evaluation = train_fn(rng)
time_elapsed = time.time() - start

sps = algo.total_timesteps / time_elapsed
print(f"Finished training in {time_elapsed:g} seconds ({sps:g} steps/second).")

Starting to train


C:\Users\jessi\AppData\Roaming\Python\Python312\site-packages\jax\_src\numpy\array_methods.py:118: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  return lax_numpy.astype(self, dtype, copy=copy, device=device)
C:\Users\jessi\AppData\Roaming\Python\Python312\site-packages\jax\_src\numpy\array_methods.py:118: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  return lax_numpy.astype(self, dtype, copy=copy, device=device)


Finished training in 4.74935 seconds (42111 steps/second).


In [665]:
print('Tuple size:', len(evaluation))
print('Episodic lengths and episodic returns shape:', evaluation[0].shape)
print(f"{algo.total_timesteps} / {algo.eval_freq} + 1 = {(algo.total_timesteps/algo.eval_freq+1)}" )

Tuple size: 2
Episodic lengths and episodic returns shape: (41, 128)
200000 / 5000 + 1 = 41.0


In [666]:
evaluation

(Array([[200, 200, 200, ..., 200, 200, 200],
        [200, 200, 200, ..., 200, 200, 200],
        [200, 200, 200, ..., 200, 200, 200],
        ...,
        [200, 200, 200, ..., 200, 200, 200],
        [200, 200, 200, ..., 200, 200, 200],
        [193, 114, 200, ..., 192, 200, 200]], dtype=int32, weak_type=True),
 Array([[-200., -200., -200., ..., -200., -200., -200.],
        [-200., -200., -200., ..., -200., -200., -200.],
        [-200., -200., -200., ..., -200., -200., -200.],
        ...,
        [-200., -200., -200., ..., -200., -200., -200.],
        [-200., -200., -200., ..., -200., -200., -200.],
        [-193., -114., -200., ..., -192., -200., -200.]],      dtype=float32, weak_type=True))

We will now be plotting the learning curve.

In [668]:
from matplotlib import pyplot as plt

episode_lengths, episode_returns = evaluation
mean_return = episode_returns.mean(axis=1)

plt.plot(jax.numpy.linspace(0, algo.total_timesteps, len(mean_return)), mean_return)
plt.xlabel("Environment step")
plt.ylabel("Episodic return")
plt.title(f"Training agent for {env_str} using {algo_str}")
plt.show()

Next, we will be looking at the `train_state`, to see what the algorithm produced.

In [670]:
jax.tree_util.tree_map(lambda x: x.shape, train_state).__dict__

{'rng': (2,),
 'env_state': EnvState(time=(1,), position=(1,), velocity=(1,)),
 'last_obs': (1, 2),
 'global_step': (),
 'last_done': (1,),
 'q_ts': TrainState(step=(), apply_fn=(), params={'params': {'Dense_0': {'bias': (1,), 'kernel': (64, 1)}, 'Dense_1': {'bias': (3,), 'kernel': (64, 3)}, 'MLP_0': {'Dense_0': {'bias': (64,), 'kernel': (2, 64)}, 'Dense_1': {'bias': (64,), 'kernel': (64, 64)}}}}, tx=GradientTransformationExtraArgs(init=<function chain.<locals>.init_fn at 0x000001EB47CC7240>, update=<function chain.<locals>.update_fn at 0x000001EB47CC6F20>), opt_state=(ScaleByAdamState(count=(), mu={'params': {'Dense_0': {'bias': (1,), 'kernel': (64, 1)}, 'Dense_1': {'bias': (3,), 'kernel': (64, 3)}, 'MLP_0': {'Dense_0': {'bias': (64,), 'kernel': (2, 64)}, 'Dense_1': {'bias': (64,), 'kernel': (64, 64)}}}}, nu={'params': {'Dense_0': {'bias': (1,), 'kernel': (64, 1)}, 'Dense_1': {'bias': (3,), 'kernel': (64, 3)}, 'MLP_0': {'Dense_0': {'bias': (64,), 'kernel': (2, 64)}, 'Dense_1': {'bias'

`RLinJAX` allows for the use of the function `evaluate` to quickly and parallelly evaluate policies

In [672]:
# Algorithms return an agent's policy parameters in its train state. 
policy = algo.make_act(train_state)
policy = jax.jit(policy)

In [673]:
from RLinJAX.evaluate import evaluate

num_seeds = 200  
env, params = gymnax.make(env_str)
max_steps = params.max_steps_in_episode

rng = jax.random.PRNGKey(0)

start = time.time()

# Evaluation time!
episode_lengths, episode_returns = evaluate(policy, rng, env, params, num_seeds, max_steps)

time_elapsed = time.time() - start

print(
    f"Evaluated {num_seeds} episodes "
    f"with a total of {jax.numpy.sum(episode_lengths)} environment steps "
    f"in {time_elapsed:g} seconds."
)

avg_return = jax.numpy.mean(episode_returns)
print(f"Average Episode Return: {avg_return}")

success_rate = jax.numpy.sum(jax.numpy.array(episode_returns) > -200) / num_seeds
print(f"Success Rate: {success_rate * 100:.2f}%")


Evaluated 200 episodes with a total of 35910 environment steps in 1.40956 seconds.
Average Episode Return: -179.5500030517578
Success Rate: 45.00%


[Text(0.5, 1.0, 'Episode return')]

In [674]:
fig, axes = plt.subplots(figsize=(8, 4), ncols=2, sharey="row")

axes[0].hist(episode_lengths, bins=10)
axes[0].set(title="Episode length", ylabel="Count")
axes[1].hist(episode_returns)
axes[1].set(title="Episode return")